In [1]:
import urllib.parse
from rdflib import Graph, Literal, Namespace, URIRef, BNode
from rdflib.namespace import RDF, RDFS, FOAF, OWL, XSD, DC, DCTERMS 
import pandas as pd

In [2]:
base_uri = "http://www.sanitasicilia.it/resource/"

g = Graph()
count = 0

sso = Namespace("http://www.sanitasicilia.it/ontology/")

g.bind("sso", sso)

ssr = Namespace("http://www.sanitasicilia.it/resource/")
g.bind("ssr", ssr)


In [3]:
def urify(ns, testo):
    testo=testo.replace(" ","_").replace("\'","")
    return ns+urllib.parse.quote(testo)

In [4]:
s = "'"
print(s)

'


In [11]:
import re
from SPARQLWrapper import SPARQLWrapper, JSON

def existsOnDbpedia(res):
    print("Richiesta per: ", res)
    sparql = SPARQLWrapper("http://dbpedia.org/sparql")

    query = """
            ASK 
            WHERE {
                ?res dbo:region dbr:Sicily .
                ?res rdfs:label ?label . 
                FILTER(LANG(?label)="it") .
                FILTER REGEX(?label, '""" + res + """').
            }
            """
    
    sparql.setQuery(query)

    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    print(results['boolean'])
    return results['boolean']

#Comuni
def addTriples(row):
    res = URIRef(urify(base_uri, row[0]))
    g.add([res, URIRef(RDF.type), URIRef(sso.Comune)])
    g.add([res, sso.hasName, Literal(row[0], datatype=XSD.string)])
    g.add([res, sso.hasTotalPopulation, Literal(row[1], datatype=XSD.integer)])
    g.add([res, sso.hasProvince, Literal(row[2], datatype=XSD.string)])
    g.add([res, sso.hasProvinceAcr, Literal(row[3], datatype=XSD.string)])


    #interlinking
    comune = row[0].replace('a\'', 'à').replace('o\'', 'ò').replace('e\'', 'è').replace('i\'', 'ì').replace('u\'', 'ù').replace('\'', '\\\''+'').title()

    if(existsOnDbpedia(comune)):
        g.add([res, RDFS.seeAlso, URIRef(urify("https://www.dbpedia.org/resource/", comune))])
    


comuni_df = pd.read_csv("../datasets/csv/completed/comuni_sicilia.csv")

comuni_df.apply(lambda row : addTriples(row), axis=1)


Richiesta per:  Alcara Li Fusi
True
Richiesta per:  Alessandria Della Rocca
False
Richiesta per:  Alì
True
Richiesta per:  Alì Terme
True
Richiesta per:  Alia
False
Richiesta per:  Alimena
True
Richiesta per:  Aliminusa
True
Richiesta per:  Altavilla Milicia
True
Richiesta per:  Aragona
True
Richiesta per:  Avola
True
Richiesta per:  Bagheria
True
Richiesta per:  Agrigento
True
Richiesta per:  Siracusa
True
Richiesta per:  Trapani
True
Richiesta per:  Palermo
True
Richiesta per:  Messina
True
Richiesta per:  Caltanissetta
True
Richiesta per:  Enna
True
Richiesta per:  Catania
True
Richiesta per:  Ragusa
True
Richiesta per:  Acate
True
Richiesta per:  Aci Bonaccorsi
True
Richiesta per:  Aci Castello
True
Richiesta per:  Aci Catena
True
Richiesta per:  Aci Sant\'Antonio
True
Richiesta per:  Acireale
True
Richiesta per:  Acquaviva Platani
True
Richiesta per:  Acquedolci
True
Richiesta per:  Adrano
False
Richiesta per:  Agira
True
Richiesta per:  Aidone
True
Richiesta per:  Altofonte
True


0      None
1      None
2      None
3      None
4      None
       ... 
381    None
382    None
383    None
384    None
385    None
Length: 386, dtype: object

In [5]:
#Farmacie

count = 0
row_prev = pd.Series()

def addBNodes(res, row):
    bn = BNode()

    g.add([res, sso.hasSite, bn])
    g.add([bn, sso.hasAddress, Literal(row[0], datatype=XSD.string) ])
   
    if(row[3] != 0):
        g.add([bn, sso.hasCap, Literal(row[3], datatype=XSD.integer) ])
    g.add([bn, sso.isIn, URIRef(urify(base_uri, row[4].title()))])
    g.add([bn, sso.tipology, Literal(row[7], datatype=XSD.string)])
    
    if(row[8] != 0 and row[9] != 0):
        g.add([bn, sso.hasLatitude, Literal(row[8], datatype=XSD.decimal) ])
        g.add([bn, sso.hasLongitude, Literal(row[9], datatype=XSD.decimal) ])

def addTriples(row):
    global count
    code = 'F'+f'{count:04}'
    res = URIRef(urify(base_uri, code))
    g.add([res, URIRef(RDF.type), URIRef(sso.Farmacia)])

    g.add([res, sso.hasName, Literal(row[1], datatype=XSD.string) ])
    g.add([res, sso.hasVatNumber, Literal(row[2], datatype=XSD.integer) ])

    addBNodes(URIRef(urify(base_uri, code)), row)
    count = count + 1


farmacie_df = pd.read_csv("../datasets/csv/completed/farmacie.csv")

farmacie_df.apply(lambda row : addTriples(row), axis=1)

g.serialize(destination="farmacie.ttl", format="turtle")

<Graph identifier=Nd029397ccc1c4f15ab8e71bd25116eb9 (<class 'rdflib.graph.Graph'>)>

In [6]:
#Parafarmacie
count = 0
row_prev = pd.Series()

def addBNodes(res, row):
    bn = BNode()

    g.add([res, sso.hasSite, bn])
    g.add([bn, sso.hasAddress, Literal(row[1], datatype=XSD.string) ])
   
    if(row[3] != 0):
        g.add([bn, sso.hasCap, Literal(row[3], datatype=XSD.integer) ])
    g.add([bn, sso.isIn, URIRef(urify(base_uri, row[4].title()))])

    if(row[7] != 0 and row[8] != 0):
        g.add([bn, sso.hasLatitude, Literal(row[7], datatype=XSD.decimal) ])
        g.add([bn, sso.hasLongitude, Literal(row[8], datatype=XSD.decimal) ])

def addTriples(row):
    global count, row_prev

    if not row_prev.empty:
        if row_prev[0] == row[0]:
            code = 'PF'+f'{count-1:04}'
            addBNodes(URIRef(urify(base_uri, code)), row)
            return
    
    code = 'PF'+f'{count:04}'

    row_prev = pd.Series(row)

    res = URIRef(urify(base_uri, code))
    g.add([res, URIRef(RDF.type), URIRef(sso.Parafarmacia)])

    g.add([res, sso.hasName, Literal(row[0], datatype=XSD.string) ])
    g.add([res, sso.hasVatNumber, Literal(row[2], datatype=XSD.integer) ])

    addBNodes(URIRef(urify(base_uri, code)), row)
    count = count + 1

parafarmacie_df = pd.read_csv("../datasets/csv/completed/parafarmacie.csv")
parafarmacie_df["LATITUDINE"] = parafarmacie_df["LATITUDINE"].apply(lambda x : float(x.replace(',', '.')) if x != '-' else 0.0)
parafarmacie_df["LONGITUDINE"] = parafarmacie_df["LONGITUDINE"].apply(lambda x : float(x.replace(',', '.')) if x != '-' else 0.0)
parafarmacie_df["PARTITAIVA"] = parafarmacie_df["PARTITAIVA"].apply(lambda x : x if x != '-' else 0)


parafarmacie_df.apply(lambda row : addTriples(row), axis=1)

g.serialize(destination="../datasets/rdf/parafarmacie.ttl", format="turtle")

<Graph identifier=Nd029397ccc1c4f15ab8e71bd25116eb9 (<class 'rdflib.graph.Graph'>)>

In [7]:
len(parafarmacie_df.index)

757

In [8]:
res = g.query('''SELECT (COUNT(?parafarmacia) AS ?cname) WHERE {?parafarmacia rdf:type sso:Parafarmacia .} ''')

for i in res:
    print(i)

(rdflib.term.Literal('742', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')),)


In [9]:
#Strutture sanitarie private
count = 0
row_prev = pd.Series()

def addBNodes(res, row):
    bn = BNode()

    g.add([res, sso.hasSite, bn])
    g.add([bn, sso.hasAddress, Literal(row[6], datatype=XSD.string) ])
   
    if(row[0] != 0):
        g.add([bn, sso.hasCap, Literal(row[0], datatype=XSD.integer) ])
    g.add([bn, sso.isIn, URIRef(urify(base_uri, row[5].title()))])

    if(row[1] != 0 and row[2] != 0):
        g.add([bn, sso.hasLatitude, Literal(row[1], datatype=XSD.decimal) ])
        g.add([bn, sso.hasLongitude, Literal(row[2], datatype=XSD.decimal) ])

    g.add([bn, sso.businessType, Literal(row[8], datatype=XSD.string)])
    g.add([bn, sso.hasSiteType, Literal(row[9], datatype=XSD.string)])

def addTriples(row):
    global count, row_prev

    if not row_prev.empty:
        if row_prev[7] == row[7]:
            code = 'SPVT'+f'{count-1:04}'
            addBNodes(URIRef(urify(base_uri, code)), row)
            return
    
    code = 'SPVT'+f'{count:04}'

    row_prev = pd.Series(row)

    res = URIRef(urify(base_uri, code))
    g.add([res, URIRef(RDF.type), URIRef(sso.StrutturaPrivata)])

    g.add([res, sso.hasName, Literal(row[7], datatype=XSD.string) ])
    g.add([res, sso.hasReferenceAsp, Literal(row[3], datatype=XSD.string)])
    addBNodes(URIRef(urify(base_uri, code)), row)
    count = count + 1

private_df = pd.read_csv("../datasets/csv/completed/private.csv")
private_df["Citta\'"] = private_df["Citta\'"].astype(str)
private_df.apply(lambda row : addTriples(row), axis=1)


0       None
1       None
2       None
3       None
4       None
        ... 
2108    None
2109    None
2110    None
2111    None
2112    None
Length: 2113, dtype: object

In [10]:
res = g.query('''
                SELECT ?den ?pfAddress
                WHERE {
                    ?str rdf:type sso:StrutturaPrivata . 
                    ?str sso:hasName ?den .
                    ?str sso:hasSite ?site .
                    ?site sso:hasAddress ?pfAddress .
                    FILTER REGEX(?den, 'ASSOCIAZIONE CASA DEI GIOVANI')
                }''')

for i in res:
    print(i.asdict())

{'den': rdflib.term.Literal('ASSOCIAZIONE CASA DEI GIOVANI', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')), 'pfAddress': rdflib.term.Literal("VIA LIBERTA' 155/A", datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string'))}
{'den': rdflib.term.Literal('ASSOCIAZIONE CASA DEI GIOVANI', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')), 'pfAddress': rdflib.term.Literal('VIA RANTERIA 20', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string'))}
{'den': rdflib.term.Literal('ASSOCIAZIONE CASA DEI GIOVANI', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')), 'pfAddress': rdflib.term.Literal('VIA MATTARELLA 55', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string'))}
{'den': rdflib.term.Literal('ASSOCIAZIONE CASA DEI GIOVANI ONLUS ', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')), 'pfAddress': rdflib.term.Literal('CONTRADA DECCARO - BAGLIO LA LOGGETTA', dat

In [11]:
#Strutture sanitarie pubbliche
import math

count = 0
row_prev = pd.Series()

def addBNodes(res, row):
    bn = BNode()

    g.add([res, sso.hasSite, bn])
   
    if(row[0] != ''):
        g.add([bn, sso.isIn, URIRef(urify(base_uri, row[0].title()))])

    if(row[1] != 0 and row[2] != 0):
        g.add([bn, sso.hasLatitude, Literal(row[1], datatype=XSD.decimal) ])
        g.add([bn, sso.hasLongitude, Literal(row[2], datatype=XSD.decimal) ])
   
    g.add([bn, sso.hasCap, Literal(row[3], datatype=XSD.integer) ])
    g.add([bn, sso.hasAddress, Literal(row[4], datatype=XSD.string)])
    g.add([bn, sso.businessType, Literal(row[7], datatype=XSD.string)])
    g.add([bn, sso.hasSiteType, Literal(row[8], datatype=XSD.string)])

    if row[9] != '':
        g.add([bn, sso.isRecoveryStructure, Literal(True, datatype=XSD.boolean)])
        g.add([bn, sso.hasRecoveryStructureName, Literal(row[10], datatype=XSD.string)])
    else:
        g.add([bn, sso.isRecoveryStructure, Literal(False, datatype=XSD.boolean)])

    g.add([bn, sso.workingMonths, Literal(row[10], datatype=XSD.integer)])
    g.add([bn, sso.openingDays, Literal(row[12], datatype=XSD.integer)])
    g.add([bn, sso.weeklyHours, Literal(row[13], datatype=XSD.integer)])
    g.add([bn, sso.hasEmail, Literal(row[15], datatype=XSD.string)])

    if row[16] != '':
        g.add([bn, sso.hasWebsite, Literal(row[16], datatype=XSD.string)])
   
    if not math.isnan(row[17]):
        g.add([bn, sso.hasPhonePrefix, Literal(row[17], datatype=XSD.integer)])
   
    if not math.isnan(row[18]):
        g.add([bn, sso.hasPhoneNumber, Literal(row[18], datatype=XSD.integer)])
    
    if not math.isnan(row[19]):
        g.add([bn, sso.hasFaxPrefix, Literal(row[19], datatype=XSD.integer)])
    
    if not math.isnan(row[20]):
        g.add([bn, sso.hasFaxNumber, Literal(row[20], datatype=XSD.integer)])

def addTriples(row):
    global count, row_prev

    if not row_prev.empty:
        if row_prev[5] == row[5]:
            code = 'SPVT'+f'{count-1:04}'
            addBNodes(URIRef(urify(base_uri, code)), row)
            return
    
    code = 'SPUB'+f'{count:04}'
    res = URIRef(urify(base_uri, code))
    g.add([res, URIRef(RDF.type), URIRef(sso.StrutturaPubblica)])
    g.add([res, sso.hasName, Literal(row[5], datatype=XSD.string) ])
    g.add([res, sso.hasVatNumber, Literal(row[14], datatype=XSD.integer)])

    addBNodes(res, row)
    count = count + 1



pubbliche_df = pd.read_csv("../datasets/csv/completed/pubbliche.csv")

pubbliche_df["Comune"] = pubbliche_df["Comune"].astype(str)
pubbliche_df["Denominazione struttura di ricovero"] = pubbliche_df["Denominazione struttura di ricovero"].astype(str)
pubbliche_df["Sito web"] = pubbliche_df["Sito web"].astype(str)

pubbliche_df.apply(lambda row : addTriples(row), axis=1)

g.serialize(destination='../datasets/rdf/sanitasicilia.ttl', 
            format='turtle')

<Graph identifier=Nd029397ccc1c4f15ab8e71bd25116eb9 (<class 'rdflib.graph.Graph'>)>

In [12]:
res = g.query('''SELECT ?name 
                 WHERE 
                {
                    ?struttura rdf:type sso:StrutturaPrivata .
                    ?struttura sso:hasName ?name .
                    ?struttura sso:isIn ?comune .
                    ?comune sso:hasName ?nomeC .
                    FILTER REGEX (?nomeC, "Palermo") .
                }
 ''')


In [13]:
for i in res:
    print(i)

In [12]:
g.serialize(destination='../datasets/rdf/sanitasicilia.ttl', 
            format='turtle')

<Graph identifier=N92196bf619e842c29584c450f0f4466f (<class 'rdflib.graph.Graph'>)>